In [ ]:
#initial alpha, K_p, sigma_x, sigma_a
mcmc = 1000
N = X.shape(0)
alpha = np.zeros(mcmc)
sigma_a = np.zeros(mcmc)
sigma_x = np.zeros(mcmc)
K_plus = np.zeros(mcmc)

#initial matrix Z
Z = np.array(np.random.choice(2,N,p = [0.5,0.5]))


alpha[0] = 1/2
sigma_a[0] = 0.5
sigma_x[0] = 1.7
K_plus[0] = 1

for i in range(1,mcmc):
    #compuate likelihood
    LH = full_X(Z,X,sigma_x,sigma_a)   
    for im in range(0,N): #loop over each image
        #sample K---num of new features
        new_k = new_K(alpha,N,Z,sigma_x,sigma_a,im)
        new_zi = np.zeros((N,new_k))
        new_zi[im,:] = np.ones((1,new_k))
        Z = np.hstack([Z,new_zi.reshape(N,new_k)]) # attach new features to Z matrix
        
        #sample new Z_i
        for k in K:
            z_sum = np.sum(Z[0:im,k])+np.sum(Z[im+1:N,k])
            Z[im,k] = full_X(Z,X,sigma_x,sigma_a)*z_sum/N
        
        
        #sample a new sigma_x with MH
        for mh in range(0,400):
            
            sig_x_prior = 1/np.random.gamma(2,2)
            
    
        #sample a new sigma_a with MH
        
    

In [2]:
#probability calculation
# X -- N*D, Z -- N*K
import numpy as np
import numpy.linalg

def full_X(Z,X,s_x, s_a):
    D = X.shape(1)
    N = Z.shape(0)
    K = Z.shape(1)
    """The constant part"""
    zz = Z.T@Z+np.diag([s_x**2/s_a**2]*K) #zz -- K*K
    determ = np.linal.det(zz)
    const = (2*math.pi)**(N*D/2) * s_x**(N*D-K*D) * determ**(D/2)
    const = 1/const
    """The exponential part"""
    ii =  np.eye(N)-Z @ np.linalg.inv(zz) @ Z.T
    tr =  np.trace(X.T @ ii @X)
    expon = math.exp(-tr/(2*s_x**2))
    return(const*expon)

In [27]:
##????? 得想想

def new_K(alpha,N,Z,s_x,s_a,obj):
    k_prob = np.zeros(10)
    for i in range(0,10):
        l = alpha/N
        new_zi = np.zeros((N,i))
        new_zi[obj,:] = np.ones((1,i))
        new_Z = np.hstack([Z,new_zi.reshape(N,i)])
        LH = full_X(new_Z,X,s_x, s_a)
        k_prob[i] = LH * (l**i)*math.exp(-l)/math.factorial(i) #likelihood*prior = posterior
        
    return(np.random.choice(10, 1, p=k_prob))

In [ ]:
def sigma()

In [31]:
Z = np.array(np.random.choice(2,10,p = [0.5,0.5]))
Z,Z[0:3]

(array([1, 0, 0, 1, 0, 1, 0, 0, 1, 1]), array([1, 0, 0]))

In [22]:
N = 5
i=3
Z = np.eye(5)
obj = 1
new_zi = np.zeros((N,i))
new_zi[obj,:] = np.ones((1,i))
new_Z = np.hstack([Z,new_zi.reshape(N,i)])
new_Z

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 1., 1., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.]])